# Divide & Conquer

* Maximum-Subarray Problem
* Inverse-Counting Problem
* Matrix Multiplication

In [1]:
import numpy as np

## Maximum-Subarray Problem

* Input: An array $A[\text{low},\ldots,\text{high}]$.
* Output: A subarray $A[i^*,\ldots,j^*]$ of $A$ such that $$\texttt{sum}(A[i^*,\ldots,j^*]) = \underset{i,j}{\texttt{argmax}}\ \texttt{sum}(A[i,\ldots,j])$$

<img src="images_02/stock-prices.png" alt="Drawing" style="width: 600px;" align="left" />

In [9]:
A = [13,-3,-25,-20,-3,-16,-23,18,20,-7,12,-5,-22,15,-4,7]

### Brute Force

* Procedure:
    * For each index $k$ from $low$ to $high$, work to the right, take and record sum at each step;
    * Find the max of the sums.
* Complexity: $O(n^2)$

In [33]:
def maxsub_brute(arr):
    len_arr = len(arr)
    l, r, subsum = 0,0,-np.inf
    for i in range(len_arr):
        for j in range(i+1,len_arr+1):
            cur_subsum = sum(arr[i:j])
            if cur_subsum>subsum:
                l,r,subsum = i,j,cur_subsum
    print 'left:',l,'| right:',r,'| subarray:',arr[l:r]
    print 'subsum:',subsum

In [34]:
maxsub_brute(A)

left: 7 | right: 11 | subarray: [18, 20, -7, 12]
subsum: 43


### DC Method

* Procedure:
    * Break the array by midpoint, and find max-subarray for each subarray;
    * For each subproblem, find the max-subarray out of (i) left array, (ii) right array, (iii) cross array.
* Complexity: $O(nlogn)$

**Subfunc: Find max cross-subarray**

<img src="images_02/find-max-crossing-subarray.png" alt="Drawing" style="width: 400px;" align="left" />

**Main/Recfunc: Find max subarray**

<img src="images_02/find-maximum-subarray.png" alt="Drawing" style="width: 500px;" align="left" />

In [74]:
def find_max_cross_subarray(arr, l, m, r):
    l_sum = -np.inf
    max_l,subsum = 0,0
    for i in range(m,l-1,-1):
        subsum = subsum + arr[i]
        if subsum>l_sum:
            l_sum = subsum
            max_l = i
    r_sum = -np.inf
    max_r,subsum = 0,0
    for j in range(m+1,r):
        subsum = subsum + arr[j]
        if subsum>r_sum:
            r_sum = subsum
            max_r = j+1 # indicating the right-bound r, which python doesn't account for in sum(arr[l:r]).
    return max_l, max_r, l_sum+r_sum

def find_max_subarray(arr, l, r):
    if l==r:
        return l, r, arr[l-1]
    mid = (l+r)/2
    ll,lr,l_sum = find_max_subarray(arr, l, mid)
    rl,rr,r_sum = find_max_subarray(arr, mid+1, r)
    cl,cr,c_sum = find_max_cross_subarray(arr, l, mid, r)
    if l_sum>=r_sum and l_sum>=c_sum:
        return ll, lr, l_sum
    elif r_sum>=l_sum and r_sum>=c_sum:
        return rl, rr, r_sum
    else:
        return cl, cr, c_sum

In [75]:
l, r, subsum = find_max_subarray(A, 0, len(A))
print 'left:',l,'| right:',r,'| subarray:',A[l:r]
print 'subsum:',subsum

left: 7 | right: 11 | subarray: [18, 20, -7, 12]
subsum: 43


## Inverse-Counting Problem

* Input: An array $A$
* Output: Count of pairs $(i,j)$ where $A[i]>A[j]$ (i.e. inverses)

<img src="images_02/inverse-counting.png" alt="Drawing" style="width: 500px;" align="left" />

In [193]:
A = [1,3,5,2,4,6]
B = [6,5,4,3,2,1]

### Brute Force

* Procedure:
    * Iterate over all the possible pairs (i.e. combinations of $i,j$) and count inverses accumulatively.
* Complexity: $O(n^2)$ (for $A,\texttt{len}(A)=n$, there are $\binom{n}{2} = \frac{n(n-1)}{2}$ pairs)

In [82]:
def invcount_brute(arr):
    len_arr = len(arr)
    inv_count = 0
    inv_pairs = []
    for i in range(len_arr-1):
        for j in range(i+1,len_arr):
            if arr[i]>arr[j]:
                inv_count += 1
                inv_pairs.append((arr[i],arr[j]))
    return inv_count, inv_pairs

In [194]:
print invcount_brute(A)
print invcount_brute(B)

(3, [(3, 2), (5, 2), (5, 4)])
(15, [(6, 5), (6, 4), (6, 3), (6, 2), (6, 1), (5, 4), (5, 3), (5, 2), (5, 1), (4, 3), (4, 2), (4, 1), (3, 2), (3, 1), (2, 1)])


### DC Method

* Procedure:
    * Piggy-back merge sort: count invs in left/right subarray, count cross-invs;
    * Combine results.
* Complexity: $O(nlogn)$

In [ ]:
"""
Subfunc: count cross-inverses

IN: arr1, arr2
i, j, inv_count = 0, 0, 0
while i,j < len(arr1), len(arr2):
    if arr1[i]<=arr2[j]:
        i += 1
    else:
        inv_count = inv_count + len(arr1)-i # i.e. there are len(arr1)-i items in arr1 that are > arr2[j]
        j += 1
OUT: inv_count

Main/Recfunc: count all inverses

IN: arr
inv_count = 0
if len(arr)>1:
    mid = len(arr)/2
    arr1, arr2 = split(arr, mid)
    l_inv_count = invcount(arr1)
    r_inv_count = invcount(arr2)
    c_inv_count = cross_invcount(arr1, arr2)
    inv_count = l_inv_count+r_inv_count+c_inv_count
OUT: inv_count
"""

In [197]:
def merge_invcount(arr1, arr2):
    len_arr1, len_arr2 = len(arr1), len(arr2)
    i,j,inv_count = 0,0,0
    arr = []
    while i<len_arr1 and j<len_arr2:
        if arr1[i]<=arr2[j]:
            arr.append(arr1[i])
            i += 1
        else:
            arr.append(arr2[j])
            inv_count += len_arr1-i
            j += 1
    if i>=len_arr1:
        return inv_count, arr + arr2[j:]
    return inv_count, arr + arr1[i:]

def invcount(arr):
    len_arr = len(arr)
    if len_arr>1:
        mid = len_arr//2
        arr1, arr2 = arr[:mid], arr[mid:]
        l_inv_count, arr1 = invcount(arr1) # count inverses in left arr
        r_inv_count, arr2 = invcount(arr2) # count inverses in right arr
        c_inv_count, arr = merge_invcount(arr1, arr2) # count cross-inverses
        return l_inv_count+r_inv_count+c_inv_count, arr
    return 0, arr

In [199]:
print invcount(A)
print invcount(B)

(3, [1, 2, 3, 4, 5, 6])
(15, [1, 2, 3, 4, 5, 6])


## More on Quick Sort

* Counting #comparisons (the dominator in running complexity) with various implementations.

In [93]:
array_path = '/home/jacobsuwang/Documents/CS TRAINING/ALGORITHMS/DATA/QuickSortArray.txt'

def get_array():
    arr = []
    with open(array_path,'r') as source:
        for line in source:
            arr.append(int(line))
    return arr

### First Elem as Pivot

In [294]:
count = 0

def partition_first(arr, l, r): 
    # l, r: left, right indices.
    piv = arr[l]
    i = l+1
    for j in range(l+1,r):
        if arr[j] < piv:
            arr[i],arr[j] = arr[j],arr[i]
            i += 1
    arr[l],arr[i-1] = arr[i-1],arr[l]
    return i-1

def quicksort_first(arr, l, r):
    global count
    if l<r:
        piv_id = partition_first(arr, l, r)
        count += r-l-1
        quicksort_first(arr, l, piv_id)
        quicksort_first(arr, piv_id+1, r)    
        
arr = get_array()
quicksort_first(arr, 0, len(arr))
print count

162085


### Last Elem as Pivot

In [302]:
count = 0

def partition_last(arr, l, r): 
    arr[l],arr[r-1] = arr[r-1],arr[l]
    piv = arr[l]
    i = l+1
    for j in range(l+1,r): 
        if arr[j] < piv:
            arr[i],arr[j] = arr[j],arr[i]
            i += 1
    arr[l],arr[i-1] = arr[i-1],arr[l]
    return i-1

def quicksort_last(arr, l, r):
    global count
    if l<r:
        piv_id = partition_last(arr, l, r)
        count += r-l-1
        quicksort_last(arr, l, piv_id)
        quicksort_last(arr, piv_id+1, r)
        
arr = get_array()
quicksort_last(arr, 0, len(arr))
print count

164123


### Median of {1st, Median, Last} Elem as Pivot

In [317]:
def partition_3median(arr, l, r):
    sublen = r-l
    m = l+sublen/2-1 if sublen%2==0 else l+sublen/2
    if arr[l]<arr[m]<arr[r-1] or arr[r-1]<arr[m]<arr[l]:
        arr[l],arr[m] = arr[m],arr[l]
    elif arr[m]<arr[r-1]<arr[l] or arr[l]<arr[r-1]<arr[m]:
        arr[l],arr[r-1] = arr[r-1],arr[l]
    else: pass
    piv = arr[l]
    i = l+1
    for j in range(l+1,r): 
        if arr[j] < piv:
            arr[i],arr[j] = arr[j],arr[i]
            i += 1
    arr[l],arr[i-1] = arr[i-1],arr[l]
    return i-1

def quicksort_3median(arr, l, r):
    count = 0
    if l<r:
        piv_id = partition_3median(arr, l, r)
        count += r-l-1
        count += quicksort_3median(arr, l, piv_id)
        count += quicksort_3median(arr, piv_id+1, r)    
    return count

arr = get_array()
print quicksort_3median(arr, 0, len(arr))